In [ ]:
!pip install -qq transformers
!pip install -qq wandb

import wandb
import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm

from torch.utils.data import DataLoader, Dataset
from transformers import AutoModel, AutoTokenizer

from nn import *
from utils import *
from sklearn.metrics import accuracy_score
from config import cfg
import gc


     |████████████████████████████████| 4.0 MB 11.9 MB/s 
     |████████████████████████████████| 6.6 MB 26.2 MB/s 
     |████████████████████████████████| 895 kB 42.3 MB/s 
     |████████████████████████████████| 596 kB 50.2 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 1.8 MB 11.3 MB/s 
     |████████████████████████████████| 181 kB 47.0 MB/s 
     |████████████████████████████████| 144 kB 52.8 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


In [ ]:
np.random.seed(cfg['SEED'])
torch.manual_seed(cfg['SEED'])
random.seed(cfg['SEED'])

In [ ]:
data = pd.read_csv('/content/mt_data_with_ner_Final.csv')
# new_data1 = pd.read_csv('/content/processed_2_4_tickers_ner3.csv')
# new_data2 = pd.read_csv('/content/processed_1_4_tickers_ner3.csv')
# new_data_pulse = pd.read_csv('/content/new_test.csv')
# test = pd.read_csv('/content/final_2.csv')

In [ ]:
# new_data=pd.concat([new_data1,new_data2])
# new_data=new_data_pulse

In [ ]:
# new_data['label'] =  new_data['label'].map({-1:2,0:0,1:1,2:2})

In [ ]:
# data = data.reset_index().fillna(' ')
# test = test.reset_index().fillna(' ')
# new_data = new_data.reset_index().fillna(' ')

In [ ]:
MODEL_NAME = cfg['model_name']
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
dl = create_data_loader(
    text=data.message,
    labels=data.label,
    tokenizer=tokenizer,
    max_len=384,
    batch_size=32,
    shuffle=False
    )

# new_data['label']=0
# new_dl = create_data_loader(
#     text=new_data.message,
#     labels=new_data.label,
#     tokenizer=tokenizer,
#     max_len=384,
#     batch_size=32,
#     shuffle=False
#     )

# test_dl = create_data_loader(
#     text=test.message,
#     labels=test.label,
#     tokenizer=tokenizer,
#     max_len=384,
#     batch_size=32,
#     shuffle=False
#     )

In [ ]:
device = 'cuda'
model = TgClassifier(3)
model.to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/diplom-tg/ruBERT-new-model/model_F.bin')['model_state_dict'])



Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

## ПРИМЕНЕНИЕ




In [ ]:
def get_labels(logits):
    softmax = F.softmax(logits,dim=1)
    float_predict = softmax@torch.tensor([0,1,-1]).numpy()
    int_predict  = softmax.argmax(dim=1).numpy()
    output = pd.DataFrame({'label_float':float_predict, 'label_int':int_predict })
    return output

#### Market Twits

In [ ]:
prediction, _ = get_predictions(model, dl)

temp = get_labels(prediction)
pd.concat([data,temp],axis=1).rename(columns={'label':'true_label'}).to_csv('labelled_mt_F.csv', index=False)

  0%|          | 0/395 [00:00<?, ?it/s]

#### Telegram (toloka)

In [ ]:
test_prediction, _ = get_predictions(model, test_dl)

temp = get_labels(test_prediction)
pd.concat([test,temp],axis=1).rename(columns={'label':'true_label'}).to_csv('labelled_tg_F.csv', index=False)

  0%|          | 0/78 [00:00<?, ?it/s]

#### Telegram (other)

In [ ]:
cleanup()
new_prediction, _ = get_predictions(model, new_dl)
temp = get_labels(new_prediction)
pd.concat([new_data,temp],axis=1).drop('label',axis=1).to_csv('labelled_new_data_F.csv', index=False)

  0%|          | 0/533 [00:00<?, ?it/s]